In [1]:
import subprocess
from datetime import datetime, timedelta
from pathlib import Path
import re
from collections import defaultdict

def run_tf_g_and_extract_fire(logfile: Path) -> list:
    try:
        cmd = f"less {logfile} | tf -g"
        result = subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.DEVNULL)
        lines = result.splitlines()
        return [line for line in lines if "FUT_TAIFEX_TXF:" in line and "[FIRE]" in line and "[FIRE DONE]" not in line]
    except:
        return []

def extract_all_fire_events(date: str) -> dict:
    """
    從當天所有 5F 與 6F 的 log 抓出 TXF 所有月份的 fire 訊息
    回傳格式：{ '202508': [ (floor, timestamp, line), ... ], ... }
    """
    base_path = Path("/nfs/datafiles.optiontraderlogs") / date.replace("-", "/")
    log_file_5f = base_path / "capital_neutrino_txf_5f" / f"output.neutrino_txf_5f.{date.replace('-', '')}.log"
    log_file_6f = base_path / "capital_neutrino_txf" / f"output.neutrino_txf.{date.replace('-', '')}.log"

    fire_lines_5f = run_tf_g_and_extract_fire(log_file_5f)
    fire_lines_6f = run_tf_g_and_extract_fire(log_file_6f)

    def parse_lines(lines: list, floor: str):
        results = []
        for line in lines:
            match = re.search(r"FUT_TAIFEX_TXF:(\d+)", line)
            if not match:
                continue
            contract_month = match.group(1)
            try:
                time_str = line[:15]
                timestamp = datetime.strptime(time_str, "%H:%M:%S.%f")
                results.append((contract_month, floor, timestamp, line))
            except:
                continue
        return results

    parsed = parse_lines(fire_lines_5f, "5F") + parse_lines(fire_lines_6f, "6F")

    # Group by contract month
    by_month = defaultdict(list)
    for contract_month, floor, timestamp, line in parsed:
        by_month[contract_month].append((floor, timestamp, line))

    # 時間排序
    for month in by_month:
        by_month[month].sort(key=lambda x: x[1])

    return by_month

def group_nearby_fires_by_month(fire_events_by_month: dict, threshold_ms: float = 0.5) -> dict:
    """
    各月份獨立進行配對
    回傳格式：{ '202508': [ [(5F, t, line), (6F, t, line)], ... ], ... }
    """
    grouped = {}

    for month, fire_events in fire_events_by_month.items():
        groups = []
        current = []
        for ev in fire_events:
            if not current:
                current.append(ev)
                continue
            diff_ms = abs((ev[1] - current[0][1]).total_seconds() * 1000)
            if diff_ms <= threshold_ms:
                current.append(ev)
            else:
                if len(current) >= 2:
                    groups.append(current)
                current = [ev]
        if len(current) >= 2:
            groups.append(current)

        grouped[month] = groups

    return grouped
def print_matched_groups_with_diff(grouped_by_month: dict):
    for month, groups in grouped_by_month.items():
        print(f"\n=== Matched FIRE for contract {month} ===")
        for group in groups:
            print("---")
            group_sorted = sorted(group, key=lambda x: x[1])  # sort by timestamp
            time_diff_ms = abs((group_sorted[1][1] - group_sorted[0][1]).total_seconds() * 1000)
            faster = group_sorted[0][0]
            slower = group_sorted[1][0]
            print(f"{faster} faster than {slower} by {time_diff_ms:.3f} ms")
            for floor, ts, line in group_sorted:
                print(f"{floor} {ts.time()} {line}")



In [2]:
from datetime import date

def process_fire_range(start_date: str, end_date: str):
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.strptime(end_date, "%Y-%m-%d").date()
    delta = timedelta(days=1)

    while start <= end:
        print(f"\n\n=== {start} ===")
        try:
            fire_by_month = extract_all_fire_events(str(start))
            matched_groups_by_month = group_nearby_fires_by_month(fire_by_month)
            print_matched_groups_with_diff(matched_groups_by_month)
        except Exception as e:
            print(f"Error processing {start}: {e}")
        start += delta


In [3]:
process_fire_range("2025-06-01", "2025-06-30")




=== 2025-06-01 ===


=== 2025-06-02 ===


=== 2025-06-03 ===


=== 2025-06-04 ===

=== Matched FIRE for contract 202506 ===
---
6F faster than 5F by 0.033 ms
6F 08:45:00.100159 08:45:00.100159 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21279 sz:1 m:1@21279x21280@13 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-15x8.25 msglag:0.000112
5F 08:45:00.100192 08:45:00.100192 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21279 sz:1 m:1@21279x21280@13 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-15x8.25 msglag:0.000136
---
6F faster than 5F by 0.023 ms
6F 08:45:00.119394 08:45:00.119394 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21277 sz:1 m:1@21277x21278@1 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-13x6.25 msglag:0.000019
5F 08:45:00.119417 08:45:00.119417 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21278 sz:1 m:2@21278x21279@1 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-14x7.25 msglag:0.000033
---
6F faster than 5F by 0.024 ms
6F 08:45:00.12

In [4]:
def summarize_fire_speed_stats(grouped_by_month: dict):
    count_6f_faster = 0
    total_6f_faster_ms = 0.0

    count_5f_faster = 0
    total_5f_faster_ms = 0.0

    for groups in grouped_by_month.values():
        for group in groups:
            if len(group) < 2:
                continue  # 跳過不成對的
            group_sorted = sorted(group, key=lambda x: x[1])
            time_diff_ms = abs((group_sorted[1][1] - group_sorted[0][1]).total_seconds() * 1000)
            faster = group_sorted[0][0]
            if faster == "6F":
                count_6f_faster += 1
                total_6f_faster_ms += time_diff_ms
            elif faster == "5F":
                count_5f_faster += 1
                total_5f_faster_ms += time_diff_ms

    print("\n===== FIRE SPEED =====")
    print(f"6F faster count: {count_6f_faster}")
    if count_6f_faster > 0:
        print(f"6F average faster: {total_6f_faster_ms / count_6f_faster:.6f} ms")

    print(f"5F faster count: {count_5f_faster}")
    if count_5f_faster > 0:
        print(f"5F average faster: {total_5f_faster_ms / count_5f_faster:.6f} ms")


In [5]:
def process_fire_range_with_stats(start_date: str, end_date: str):
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.strptime(end_date, "%Y-%m-%d").date()
    delta = timedelta(days=1)

    all_groups_by_month = {}

    while start <= end:
        try:
            fire_by_month = extract_all_fire_events(str(start))
            matched_groups_by_month = group_nearby_fires_by_month(fire_by_month)
            # 合併到總結果中
            for month, groups in matched_groups_by_month.items():
                all_groups_by_month.setdefault(month, []).extend(groups)
        except Exception as e:
            print(f"Error processing {start}: {e}")
        start += delta

    summarize_fire_speed_stats(all_groups_by_month)


In [6]:
process_fire_range_with_stats("2025-06-01", "2025-06-30")



===== FIRE SPEED =====
6F faster count: 22
6F average faster: 0.026182 ms
5F faster count: 2
5F average faster: 0.038500 ms
